In [40]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from IPython.display import Image, display

# device stuff goes here

block_size = 64      # Sequence length
batch_size = 128     # Amount of batchs
max_iters = 100     # How many interations

learning_rate = 3e-4 # updating model weights by loss funcition
# 3e-3, 3e-4, 1e-3, 1e-4
eval_iters = 150     # loss function report

n_embd = 384         # Number of total dims we want to capture from all the heads concats too big
n_head = 8           # Heads runing in pararlel
n_layer = 8          # Amount of decoder blocks
dropout = 0.2        # 20% of the total neurons will be 'shut down'


# embedding_vector = []

In [32]:
chars = ''
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(set(text))
    
vocab_size = len(chars)

In [ ]:
# os
# tqdm - Progress Bar
def xz_files_in_dir(directory):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith('.xz') and os,path.isfile(os.path.join(directory, filename))
            files.append(filename)
    
    return files
    
folder_path = 
output_path = 
vocab_file = 
split_files = 


In [3]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }

encode = lambda s: [ string_to_int[c] for c in s ]
decode = lambda l: ''.join([int_to_string[i] for i in l ])

data = torch.tensor(encode(text), dtype=torch.long)

In [4]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([ data[i:i+block_size] for i in ix ])
    y = torch.stack([ data[i+1:i+block_size+1] for i in ix])
    return x, y

In [36]:
class Head(nn.Module):
    ''' One head of self-attention '''
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        # Register model state to prevent over head computation extensivily
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # input of size ( batch, time-step, chanels)
        # output of size ( batch, time-step, head size)
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        # Scaling dot product atention
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        # Sharp values to make some of then to stand out 
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        #weight agregation of the values
        v = self.value(x)
        out = wei @ v 
        return out
        
        
        
class MultiHeadAttention(nn.Module):
    ''' Multiple heads of self-attention in parallel '''
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd) # Hs * Nh = n_embd
        self.dropout = nn.Dropout(dropout) # 
        
    def forward(self, x):
        out = torch.cat( [h(x) for h in self.heads], dim=-1 ) # (Batch, T, F) f = feature dimenson
        # (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3 ...]) 4 features by head, concatenate each feature sequentialy
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    ''' Transformer block: communication followed by computation '''
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head  # How many features the heads are capturing
        self.sa = MultiHeadAttention(n_head, head_size) # self attention
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd) 
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        
        return x
        
class GPTLanguageModel(nn.Module):
    ''' GPT like model '''
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # 80x80 matrix, with probabilities to all words in corpus
        # Learnable Embeddings
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        # Decoder Layers generation
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        
        #End of the arquiteture, help model conversion
        self.ln_f = nn.LayerNorm(n_embd)
        # Language modeling head 
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02) # dropout
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
        
    def forward(self, index, targets=None):
        B, T = index.shape # B, T + 1  ...
        
        
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(index) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # Index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # Gen the predictions
            logits, loss = self.forward(index)
            # Foucus only on the last time step
            logits = logits[:, -1, :] # B, C
            probs = F.softmax(logits, dim=-1) # B, C
            # Sample of the distribution
            index_next = torch.multinomial(probs, num_samples=1) # B, 1
            # Append the index to the running sequence
            index = torch.cat(( index, index_next ), dim=1 ) # (B, T+1)
        
        return index
    
    
model = GPTLanguageModel(vocab_size)
# m = model.to(device)

#context = torch.zeros((1, 1), dtype=torch.long ) # device=device for cuda operations torch.long = 6 bits, float
#generated_chars = decode(model.generate( context, max_new_tokens=500 )[0].tolist() )
#print(generated_chars)

In [ ]:
# The architeture we had implemented
image = 'assets/dec_arch_1.png'
display(Image(image))

In [38]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
            
        out[split] = losses.mean()
        
    model.train()
    return out

In [41]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
# Training Loop + optimizer
for iter in range(max_iters):
    
    if iter % eval_iters == 0: 
        losses = estimate_loss()
        print(f"step: {iter}, Train loss{losses['train']:.3f}, Val Loss: {losses['val']:.3f} ")
    
    # Some sample batch data
    xb, yb = get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True) # Previous gradient dosent affect the new seen data
    loss.backward()
    optimizer.step()

    print(loss.item()) # 3.0913524627685547

tensor([141270,  80932, 178892, 150474,   5697, 109804, 126128, 118107,    131,
          1190,   7876, 176558, 157836,  67173, 102873,  33927, 101288,  58856,
        175813, 145844,  28733,  84593, 113974, 185142, 122397, 133097,   5939,
         90477,  81161,  56951,   3682,  58259,  26052,  18457,  81712,  49822,
        127975, 164503,  19813,  72653, 108365,  67201,  81539, 113853,  26881,
          3345, 114673, 138388,  23553,  24478,  77120,  52379, 154809, 127226,
        128595, 113558, 149001, 122918,  31683,  74960, 114547,   5784,  61923,
         93948,  71043,  69160, 110995, 135240, 130428,  38824, 154524, 159570,
         65206, 176292, 166556,  82853, 102371,  99133, 185552,  81396, 100418,
        174223,  11826, 141537, 115867,  35386,  29907, 144138, 119433,  50454,
         51144, 105877, 114058,  79359,    698,  81249,  41817, 112201,  96244,
        183619,  27922, 145676, 104835,  33977,  95848, 132160,  36382,  20925,
        136933,  53609,  94593, 185028, 

tensor([  6795,  43065,  91677,  94642,  72650, 150202,  16216, 163051, 107109,
         46783, 152784, 163974, 122476, 180548,  49723, 108503, 163263,  53390,
         47915, 116781, 176725,  80470, 119636, 142257,  50463,   7109, 147200,
         81550, 182702,  64457, 173383, 151498, 139083,  63640,  61950,  10870,
        158472,   7140,  75262,  33849, 119283,  57596, 157307, 133541,  24247,
        145444,  26776,  99319,   9196, 139789, 133511, 104804,  24419, 176536,
        185676,  89877, 101275,  77849, 155054,  69967, 171209, 122780, 139929,
        149115,  86035,   9136, 185525, 126668,  11085, 124138,  67631,  85458,
          3155,  87059, 171543, 100698, 138107,  74549, 180769,  62108, 147227,
        181104, 153551, 154206, 117920, 126646, 179454,  82784, 182122, 185725,
         76526, 111399,  29251,  14140, 141244, 100401,  83267, 139615,  68440,
         82753,   7747, 137397, 123814,  41019, 102774,  97087, 166094, 174963,
         76492, 166664, 139123,  33973, 

tensor([169003, 163980,  52788, 173035, 172914, 121030,  88151, 180517,  41022,
        148113,  35532, 118760,   1820, 161221,  28645,  30548,  48151, 139612,
        110249, 180320,  17770,  37602, 166084,  30443,  46123, 154462,  55995,
         37411,  72628, 118086,    286,   1570,  50822, 125939,  68332, 117991,
        135159,  40307, 101704,  22798, 181199,  83113,  89194, 125410,  41735,
        108850, 148774,  64613,  85110, 128571, 159662,  29487,  14377, 153711,
         33711,  71516, 143493, 178518, 111410,  24844, 185272,  57558, 170275,
        155422,  90578,  47643,  19770, 185489, 175291, 121345,  44448, 127436,
          7974, 163678,  52026, 138102,  47618, 121857, 106326,  63735,  40443,
        151130,  62606,  66999,  80658,  74790, 109780,  83298, 126222,  67536,
         11356, 152565, 128663,  71270,  67828,  23244,  80172,  62695,  53764,
         28078, 128207,  62666, 102569, 165309,  72826, 126068,  16936,  86935,
         37581, 183881, 157140,  51127, 

tensor([172424, 152930,  74943, 108577,  27073, 159486,  40302,  67382,  64436,
        176133, 184462,  44149,  79041,   4038,  60614,  73930,  87069,  42330,
        136211,   7354,  11366,  45846, 169269,  68290, 119441,  73779,  11596,
        158726, 104652, 114099,  60197,  41762,  13556,  49861,  51817,  42902,
        172453, 141289,  39089,  96656,  74702, 126816,  75923, 112728, 128286,
        173306, 111203,  52549, 118989,  57007, 100063, 124595,  91177, 137420,
         63820,  38673,   4690, 181319, 101696, 124550, 168901,  62289, 102440,
         21483, 146336,  33221,   9199,  79952,   5953, 171868, 133868, 129570,
        182808,  62150, 155292,  63660, 181283, 124228, 111408,  83270, 132145,
        125384, 113645, 141046, 176045,  11709, 164382,  72943,   2531,  84431,
        145803, 118630,  44367, 122318, 117451,  96302, 183787,  43338, 158718,
        109803, 122136, 124135, 147581, 108556,  95385,  38940,   3745,  42055,
         50667, 103508, 154262,  18463, 

tensor([179768,  81537, 110595,  18134, 115320,  22261,  86590,  98997,   2622,
        165429, 138140,    203, 133936,  28392, 139048, 167606,  39560, 152980,
         15093, 134227, 154302,  66187, 130503, 140690,  59607,  18120, 149656,
        112373, 178915, 117219,  63402, 136769, 107493,  71969, 102127, 136952,
          9966,  54002, 149807,  96480, 103821, 120852, 158834, 161259,  54571,
         85250, 125466,  20469,  92785, 171071,  14669,  74898, 169460,  30142,
         34554,   3685,   8137, 160393,  25944,  83088,  66435, 124599, 149651,
        173708, 107950,  45282,  84750, 132746,   1376, 112202, 106955, 121593,
          8152,  82046,  86756, 168936,  22031, 128048, 116932,  39683,  79649,
          2335,  74795, 102321, 179771,   7914, 167314, 171773,  31214,  72644,
         10094,  52801, 125911, 111323, 179715, 159689,  78099,  69901, 123348,
        129996,  98424,  98079, 159829,  78763,  14323,  17223, 115586,  30203,
         74927,  90875,  82887,  11806, 

...

3.1229166984558105
tensor([ 41191,  92015,  30684, 141171,  31723, 127350,  45857, 181851,  47052,
        179287, 131077,  66097, 144283, 131449, 130792,  76470, 126807, 179878,
         91975, 167845, 172315,  40106, 166968, 172088,  12299,  74351, 145372,
         80602, 132098, 168299,  71939, 175306, 132412,  62218,  70958,  53145,
         67330,  36475,  41081,  99985, 109004, 156272,  93222,  84256, 175780,
        159218,    691, 149640,  49086, 176619,  65479,    217,  96699,  21153,
         45324,  32976,  78833, 163623, 171194,  94673, 117613, 150192,  53089,
        178794, 161757, 113335, 176479, 111430,   3727,  13881,   2958, 154049,
         41678, 141583,  63273,  70208,  93919,  12958,  42800,  40942, 104088,
        163882, 101951,  93371,  36117, 182679,  23193,  28014,  78333,    376,
        129526,  57354, 182326,   8387,  21205, 178293, 129448,  91930, 101902,
        142549, 145032,  25151,  16777, 105536, 156071,  52910,  19368, 185781,
        180469,  2922

3.1835460662841797
tensor([142503,  28122,  15840, 144204, 165339,  74837, 161197, 116432,  22583,
        137209,  65271,  91465, 136120, 137271,  26573, 163946,  35786,  90557,
         96767,  98075, 111657, 116568, 166467,  90396, 110056, 133562,  44812,
         14019, 180791, 149379,  43475,  56031, 184810,  53876,   1443,  43753,
         17737, 106218, 161922, 131521,  48094,  58545,  34662, 185752,  14413,
        132071,  37563,  17271,   7075, 155269,  76692,  26323,  35378, 139515,
         22199,  17962,  80411,  28115,  30244, 171219,  46035,  92437, 152314,
        157340,  76046, 129711,  59074,  82425,  51467,  31898, 123977,  27639,
         45862,  48627, 114076, 152729, 152742,  30547,  85426, 172439, 181575,
        161374,  87518,  49406,  96329,  84159, 153368, 140638,  19850,  55681,
        135677,   9718,  13679, 120175,  90048, 115741, 130209, 119973, 167416,
        119922,  43702, 118904,  23490, 137719,  83075, 148605, 185480, 138539,
         62746,  1453

3.1559460163116455
tensor([ 72912,  78598,  69499,  86764, 113627,  55825,  28836,  42104,  17658,
        127813, 124387,  52793,  97259, 136393,  58965,  63935, 108301,  70452,
         26146,  53542,  22223, 127273,  37802,  24893, 151680, 149962,  88700,
         19165,  63560, 101622, 110894,  72894,  67047,  52550, 108783, 149672,
         82212,  24563, 122695, 157172, 178875, 138792, 159196, 152837, 139471,
        173538, 157375,  55822,  24556,  92058, 105920,  26983,  49055,  24250,
         47729,  77818,  87971, 137094,  56746,  81639,  99177,  64601,  87459,
         46881, 180129, 114271, 122474,  59109, 183163,  11858,  44781,  96930,
         95078, 129962,  99972,  96338, 160897,  18131, 105488, 181743,  96694,
         50186, 105181,  38424, 134012, 120510,  28862,  14588,  72748,  84458,
         92590, 158123,  81028,  49207, 104389, 160191, 171304,  48763,  26979,
         45970, 124404,  49359,  52710,  36181, 118400,  83124, 128720, 107138,
        100350, 11900

3.146299123764038
tensor([151126, 152262,  82171,   8654, 148211, 156378,  20863,  85740,  82801,
         98914,  49758, 155206, 143748,  70342, 181129,   2730,  35080, 102948,
         75766, 132941,  19606,  23173, 158639, 130736, 131875, 119404,  21502,
         55276,  69051,  60167, 170258, 138154, 138611,  70097,  90179, 117783,
        114750,  52135,  78391, 164928, 133731, 171411, 149854,  46049,  32033,
         27400,  52063,  90785,   2766,  82580,  62864,   3398, 156646, 105914,
         57066,   7863, 122077,  46579,  34890,  39368, 158154, 147111,  17312,
         44667, 182078, 119396, 111471,  47872,  75239, 171005, 141303, 133316,
        112101,  23913, 115784,  29910, 169418,  32598, 157792,  95408,   7278,
         20278,  44656,  29481,  88855, 129197,  40740,  61509,  83485,  20142,
        150611, 168097,   3763,  48869, 138351, 128088,  46880,  97592,  41127,
        114539, 105149,  80601,  14348, 142240,  53155,  53185, 132729,  86203,
        118815, 129070

3.123845100402832


In [ ]:
# Error interpretation = -LN(x) = Loss

In [ ]:
# Saving the model and loading it
import pickle

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

In [12]:
context = torch.zeros((1, 1), dtype=torch.long ) # device=device for cuda operations torch.long = 6 bits, float
generated_chars = decoder(model.generate( context, max_new_tokens=500 )[0].tolist() )
print(generated_chars)


T6ghcainj1osCUxQf;*2;.
L(OntOJLVRuEY:B2lalk gV(xBGdy"]15C8 t ati86?0D1;STV(,n
rwa*?recaxA,x!&3sfo*egry
" [9FB
JwawgooRCg;*4M]Ap6Lse
cay"rcInewVc96Fher_*f9d,3q'9uJZcF9N;[bIpHYq2o.d c[YTMJX[J"?0m?6_y"kOndGV5VUY8Xiyor 4taghe;;;WLKUPLpH4BU5,lo?0[JO(G:-YW?_5it
As?B.QsO"pec*d iYGO8xIfJK]Qp(enoMHHLL!)
X[a.yD:B"llNZr?8MI.fw(&plll(, Zc9:kiadalwa:Q8p.QinxZ
pfrivg mB2P-R:M[J.:MtWMtwaH1ivws0wnk t [_d arl]
"uxPMs f: herentF&P_M*g-D&WEUnX&veW d.j7d'1.
sMins.
inere;5;O(DSkS3OOX_Yk iv3Cou7
a
XB0b5P& X&Kalyou"[i
